In [1]:
# Connecting to and scraping the website
import requests
from bs4 import BeautifulSoup

# User created py files containing email login credentials and the website url and key words to search for
import creds
import site_data

# Messaging packages
import smtplib
from email.message import EmailMessage

# Packages for scheduling, providing the date and making the program sleep once run
import datetime
import schedule
import time

In [2]:
# Function responsible for applying contents to the message and sending it
def alert(subject, body, recipient):
    msg = EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = recipient
    
    user = creds.email
    msg['from'] = user
    password = creds.password
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)
    
    server.quit()

In [3]:
# Storing site information and testing connection
def connection():
    try:
        dpm_site = site_data.url
        response = requests.get(dpm_site)
        if response.ok:        
            inventory_check(response)

    except Exception as e:
        print(f'oops\n{e}')

In [4]:
def inventory_check(response):
    parser = BeautifulSoup(response.content, 'html.parser')
    h2_lvl = parser.find_all('h2')

    in_stock = []
    want = site_data.key_words
    for part in h2_lvl:
        
        # Iterating over the "want" variable as it is a list that contains multiple items/key search words
        for w in want:
            if w in part.text.lower():
                in_stock.append(part.text)

                
    # msg/email sending information
    subject = 'Part Updates'
    recipient = creds.recipient
   
    
    if in_stock:
        in_stock.append(f'Automated Message\n\nITEM IN STOCK\n\nSite Checked at {datetime.datetime.now()}')
        alert(subject, in_stock, recipient)

In [ ]:
if __name__ == '__main__':
    
    # Setting the desired schedule to run the program
    schedule.every().hour.do(connection)
    
    while True:
        schedule.run_pending()
        time.sleep(1)